In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType

from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import udf

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/04 23:36:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#df = pd.read_csv("dataset-problemset2-ex5/user_artist_data_small.txt", delim_whitespace=True, names=['user_id', 'artist_id', 'play_count'])

#df

schema = StructType([
    StructField("user_id", IntegerType(), False), \
    StructField("artist_id", IntegerType(), False), \
    StructField("play_count", IntegerType(), True)\
        ])
    

df = spark.read.csv("dataset-problemset2-ex5/user_artist_data_small.txt", sep=' ', schema=schema)
df.printSchema()
df.show()

root
 |-- user_id: integer (nullable = true)
 |-- artist_id: integer (nullable = true)
 |-- play_count: integer (nullable = true)

+-------+---------+----------+
|user_id|artist_id|play_count|
+-------+---------+----------+
|1059637|  1000010|       238|
|1059637|  1000049|         1|
|1059637|  1000056|         1|
|1059637|  1000062|        11|
|1059637|  1000094|         1|
|1059637|  1000112|       423|
|1059637|  1000113|         5|
|1059637|  1000114|         2|
|1059637|  1000123|         2|
|1059637|  1000130|     19129|
|1059637|  1000139|         4|
|1059637|  1000241|       188|
|1059637|  1000263|       180|
|1059637|  1000289|         2|
|1059637|  1000305|         1|
|1059637|  1000320|        21|
|1059637|  1000340|         1|
|1059637|  1000427|        20|
|1059637|  1000428|        12|
|1059637|  1000433|        10|
+-------+---------+----------+
only showing top 20 rows



a) Populate a utility matrix. Be sure to first replace bad artist ids that are due to known misspellings using the assignments in artist_alias_small.txt. Think about how to store the matrix in a reasonable way.

In [41]:
# replace bad artists e.g. 6979261

schema_artists = StructType([\
    StructField("misspelled_name", IntegerType(), False),
    StructField("correct_name", IntegerType(), False)])

false_to_correct_artists = spark.read.csv(
    "dataset-problemset2-ex5/artist_alias_small.txt", sep='\t', schema=schema_artists)

false_to_correct_artists.printSchema()

false_to_correct_artists.show()

#false_to_correct_artists.show()


#return from misspelled artist the correctly speller artist. I assume that "artist_alia_small.txt" is in the format <misspelled artist> <correctly spelled artist>
def get_correct_artist(misspelled_artist: int, false_to_correct_artists):
    df = false_to_correct_artists.filter(
        misspelled_artist == false_to_correct_artists[0]).collect()

        # if no artist was returned then the artist was already correctly spelled
    if len(df) == 0:
        return misspelled_artist
    else:
        correct_artist = df[0][1]
        return correct_artist


# pack above method in a udf, with the hope that the map function will run without bugs. it did't work :(
correct_artist = udf(lambda x: get_correct_artist(
    x, false_to_correct_artists), IntegerType())

#the actual mapping in the second column from misspelled to correctly spelled
rdd_clean = df.rdd.map(lambda x: (x[0], correct_artist(x[1]), x[2]))


columns = ['user_id', 'artist_id', 'play_count']


# transforming rdd to dataframe causes error that i cant solve
#df_clean = rdd_clean.toDF(columns)


root
 |-- misspelled_name: integer (nullable = true)
 |-- correct_name: integer (nullable = true)

+---------------+------------+
|misspelled_name|correct_name|
+---------------+------------+
|        1027859|     1252408|
|        1017615|         668|
|        6745885|     1268522|
|        1018110|     1018110|
|        1014609|     1014609|
|        6713071|        2976|
|        1014175|     1014175|
|        1008798|     1008798|
|        1013851|     1013851|
|        6696814|     1030672|
|        1036747|     1239516|
|        1278781|     1021980|
|        2035175|     1007565|
|        1327067|     1308328|
|        2006482|     1140837|
|        1314530|     1237371|
|        1160800|     1345290|
|        1255401|     1055061|
|        1307351|     1055061|
|        1234249|     1005225|
+---------------+------------+
only showing top 20 rows



In [42]:
get_correct_artist(1002670, false_to_correct_artists)


1034635